In [33]:
#librerias
import numpy as np

In [34]:
#Integración numerica
def n_integral(f,n,a,b):
    valor=0
    for k in range (1,n):
        valor=valor + f(a+k*(b-a)/n)
    valor=valor+(f(a)+f(b))/2
    valor=valor*(b-a)/n
    return valor

Problema 1: 

(1) Implementación

a)

In [35]:
n=10 #numero de discreticazión (modificable)
a=0 #Inicio del dominio (modificable)
b=1 # final del dominio (modificable)
def f(x): 
    #Función del problema (modificable)
    return 3*x**2

In [36]:
def xi(i):
    return a+i*(b-a)/n

In [37]:
def phi(x):
    if x>= xi(i-1) and x<= xi(i):
        return (x-xi(i-1))/(xi(i)-xi(i-1))
    if x>=xi(i) and x>=xi(i+1):
        return (xi(i+1)-x)/(xi(i+1)-xi(i))
    else:
        return 0
    
def derivada_phi(x):
    if x>= xi(i-1) and x<= xi(i):
        return 1/(xi(i)-xi(i-1))
    if x>=xi(i) and x>=xi(i+1):
        return -1/(xi(i+1)-xi(i))
    else:
        return 0

def phi_anterior(x):
    if x>= xi(i-2) and x<= xi(i-1):
        return (x-xi(i-2))/(xi(i-1)-xi(i-2))
    if x>=xi(i-1) and x>=xi(i):
        return (xi(i)-x)/(xi(i)-xi(i-1))
    else:
        return 0
    
def derivada_phi_anterior(x):
    if x>= xi(i-2) and x<= xi(i-1):
        return 1/(xi(i-1)-xi(i-2))
    if x>=xi(i-1) and x>=xi(i):
        return -1/(xi(i)-xi(i-1))
    else:
        return 0

def phi_superior(x):
    if x>= xi(i) and x<= xi(i+1):
        return (x-xi(i))/(xi(i+1)-xi(i))
    if x>=xi(i+1) and x>=xi(i+2):
        return (xi(i+2)-x)/(xi(i+2)-xi(i+1))
    else:
        return 0
    
def derivada_phi_superior(x):
    if x>= xi(i) and x<= xi(i+1):
        return 1/(xi(i+1)-xi(i))
    if x>=xi(i+1) and x>=xi(i+2):
        return -1/(xi(i+2)-xi(i+1))
    else:
        return 0

def ai(x):
    return derivada_phi(x)**2+phi(x)**2
def bi(x):
    return derivada_phi(x)*derivada_phi_anterior(x)+phi(x)*phi_anterior(x)
def bi_superior(x):
    return derivada_phi(x)*derivada_phi_superior(x)+phi(x)*phi_superior(x)
def fi(x):
    return f(x)*phi(x)

In [38]:
M=np.zeros((n-1,n-1)) #Matriz en los que pondremos los valores de nuestro sistema
F=np.zeros((n-1,1)) #Vector de valore de f_i
for i in range(1,n):
    F[i-1,0]=n_integral(fi,20,xi(i-1),xi(i+1))
    M[i-1,i-1]=n_integral(ai,20,xi(i-1),xi(i+1))
    if i!=1:
        M[i-1,i-2]=n_integral(bi,20,xi(i-1),xi(i))
    if i!= n-1:
        M[i-1,i]=n_integral(bi_superior,20,xi(i),xi(i+1))


In [39]:
F

array([[0.0009075],
       [0.0048675],
       [0.0094275],
       [0.0226875],
       [0.0365475],
       [0.0537075],
       [0.0594675],
       [0.0979275],
       [0.1249875]])

In [40]:
#Método de Cholesky
L=np.zeros((n-1,n-1)) #Matriz en los que pondremos los valores de nuestro sistema
L[0,0]=M[0,0]**(1/2)
suma=0
suma_diag=0
for i in range(1,n-1):
    for j in range (0,i):
        suma_diag=suma_diag +L[i,j]**2
        for k in range (0,j):
            suma=suma +L[i,k]*L[j,k]
        L[i,j]=(M[i,j]-suma)/L[j,j]
        suma=0
    L[i,i]=(M[i,i]-suma_diag)**(1/2)
    suma_diag=0    

i
1
j
0
i
2
j
0
1
i
3
j
0
1
2
i
4
j
0
1
2
3
i
5
j
0
1
2
3
4
i
6
j
0
1
2
3
4
5
i
7
j
0
1
2
3
4
5
6
i
8
j
0
1
2
3
4
5
6
7


In [41]:
L

array([[ 3.32242381e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  3.32242381e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.13862763e-17,  3.16678070e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  4.48747674e-17,
         3.32242381e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -4.27725525e-17,  3.32242381e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  3.322423

In [43]:
#Resolveremos ahora Ly=F
y=np.zeros((n-1,1))
y[0,0]=F[0,0]/L[0,0]
suma=0
for i in range(1,n-1):
    for j in range(0,i):
        suma=suma+L[i,j]*y[j,0]
    y[i,0]=(F[i,0]-suma)/L[i,i]
    suma=0


In [44]:
y

array([[0.00027314],
       [0.00146504],
       [0.002977  ],
       [0.0068286 ],
       [0.01100025],
       [0.01616516],
       [0.01877853],
       [0.02947472],
       [0.03761937]])

In [54]:
#Resolveremos ahora  L^{T}x=y
LT=np.transpose(L)
x=np.zeros((n-1,1))
x[n-2,0]=F[n-2,0]/L[n-2,n-2]
suma=0
for i in reversed(range(1,n-1)):
    for j in reversed(range(i,n-1)):
        suma=suma+L[i,j]*x[j,0]
    x[i,0]=(F[i,0]-suma)/L[i,i]
    suma=0

In [55]:
x

array([[ 0.00000000e+00],
       [ 1.46504488e-03],
       [ 2.97699806e-03],
       [ 6.82859902e-03],
       [ 1.10002522e-02],
       [ 1.61651562e-02],
       [ 1.87785343e-02],
       [ 2.94747165e-02],
       [-4.17700708e-18]])